In [1]:
#_*_encoding:utf-8_*_
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import re
from selenium.webdriver.common.by import By
import datetime
import pandas as pd
import numpy as np

In [2]:
driver = webdriver.Chrome('C:/D\中山/31lab/裕隆日產/chromedriver.exe')

In [3]:
driver.close()

#### 開啟&登入

In [2]:
driver = webdriver.Chrome('./chromedriver')
driver.get("https://mbasic.facebook.com/groups/internlens/?refid=46&__xts__%5B0%5D=12.Abpf2d6pkijgmhRLt7n7t1TRb5vDnFWgmb0EL4XLVvTqn6ac0M8i3OCL1uXgyUAqDmUQ1prkYzWTRC00J5wkfJ2QShJoh8PmcfM5lSlO4gGVW_yeUAF6SAzsb16HeC35Cpp-GlHhZLdxM_nez3BTyy8fpzXBc5clx7CFbTccLJYnzJoGRZCbG32pZRnTgjXJDpixDkKdMCEKHwGkqa3jBWXQmuCNp5L4auZAIR02UEmT8WaVi5llQt-ut2GFNqDf4NRJ0unf73y9v0Ah6rVAwV_NzIG47oH1vIHeG88awtCPJAenpOz8fpbS_IXrvivUufX7oY7AB0gqeZvp_qlPp-AWjKrap4NSJYQRq8_HTEJPztepiAQ5tmiTS8Y-NkLiYCfo-oi7qbeJQ9sWmIEHl1AtHe8t7piv3ckq3x_oDa7ZkTbl8e0bYS-a-RS1wxYn1Z7Tr0XLSYyn446cqQj9GbJ8NVRvsWdNpbiIQJzwrNAjzuixSjcaWBwhik7ao1xJ5A1GklNd-Wu49SAnek9QuuKRcHw2u0j1oPoPrmyIHZfx4z8HohwHQ3sb1X34Cr566fQ&paipv=0&eav=AfbFF1e-i2bOSDdpcF-pr4g_4urOZ59CMpmhF99mlnN9lPQJ4uoCl9ilr_PTnFEs4yQ")
print(driver.title)

登入 Facebook | Facebook


In [3]:
account_text = driver.find_element(By.XPATH, '//*[@id="m_login_email"]')
time.sleep(2)
account_text.send_keys('cindy96515@yahoo.com') # 輸入帳號
pwd_text = driver.find_element(By.XPATH, '//*[@id="password_input_with_placeholder"]/input')
time.sleep(2)
pwd_text.send_keys('cindy0325') # 輸入pwd
login_btn = driver.find_element(By.XPATH, '//*[@id="login_form"]/ul/li[3]/input')
time.sleep(1)
login_btn.click()

In [6]:
print(driver.title)
url = "https://mbasic.facebook.com"

工作與實習 Jobs and Internships


#### 爬文章連結&meta data

In [7]:
resp = driver.page_source
soup = BeautifulSoup(resp,"html.parser")
articlesRoot = soup.find_all('div', id='m_group_stories_container') #文章最上層
articles = (articlesRoot[0].find_all("article", class_ = "da dc dm")) #articles per page
articles

[<article class="da dc dm" data-ft='{"qid":"-4211482749287533861","mf_story_key":"2422616467900235","top_level_post_id":"2422616467900235","tl_objid":"2422616467900235","content_owner_id_new":"100001577485398","page_id":"501424373352797","src":22,"photo_attachments_list":["6371982842864235","6371983166197536"],"photo_id":"6371982842864235","story_location":6,"story_attachment_style":"album","filter":"GroupStoriesByActivityEntQuery","ott":"ottAX9AknnncQ_asoQ8","sty":308,"ent_attachement_type":"PhotoSetAttachment","page_insights":{"501424373352797":{"page_id":"501424373352797","page_id_type":"group","actor_id":"100001577485398","dm":{"isShare":0,"originalPostOwnerID":0},"psn":"EntGroupMallPostCreationStory","post_context":{"object_fbtype":657,"publish_time":1683269036,"story_name":"EntGroupMallPostCreationStory","story_fbid":["2422616467900235"]},"role":1,"sl":6}},"actrs":"100001577485398","tds_flgs":3,"ftmd_400706":"111114l","tn":"-R"}' id="u_0_3_xB"><div class="dn"><header class="do"><

In [52]:
# link_list = []
count_list =[]
result = []
for i in range(500):
    resp = driver.page_source
    soup = BeautifulSoup(resp,"html.parser")
    articlesRoot = soup.find_all('div', id='m_group_stories_container') #文章最上層
    articles = (articlesRoot[0].find_all("article", class_ = "da dc dm")) #articles per page
    for article in articles:
        count = 0
        data = {}
        # tmplink = article.select(".dn .ds a:not([class]):not([data-ft])")[-1:][0]['href']
        try:
            tmplink = article.select(".dn .ds a:not([class]):not([data-ft])")[-1:][0]['href']
        
            if("mbasic" not in tmplink):
                tmplink = 'https://mbasic.facebook.com'+tmplink
        except:
            tmplink = None
        # likes = int((article.find("footer").find_all("div", class_ = "cp cq")[-1:][0].select("a[aria-label]"))[0].text)
        like_str = article.find("footer").find_all("div", class_ = "cp cq")[-1:][0].find("span", class_ = "cp cq").text
        like_str = re.findall(r'\d+', like_str)
        if(len(like_str) > 0) : 
            like_str=int(like_str[0]) 
        else: 
            like_str = 0
        # 留言
        comment_num = (article.find("footer").find_all("div", class_ = "cp cq")[-1:][0].select('a.dv:not([class*=" "])'))[0].text
        comment_num = re.findall(r'\d+', comment_num)
        if(len(comment_num) > 0) : 
            comment_num=int(comment_num[0]) 
        else: 
            comment_num = 0
        data['Url'] = tmplink
        data['likes'] = like_str
        data['comment_num'] = comment_num
        if tmplink!=None:
            count+=1
            result.append(data)
    count_list.append(count)
    driver.find_element(By.PARTIAL_LINK_TEXT,"查看更多貼文").click()

In [57]:
pd.DataFrame(result).to_csv("Job_links_likes.csv",encoding='utf-8-sig')

In [58]:
df_link = pd.DataFrame(result)

### 讀取所有link csv

In [8]:
df = pd.read_csv("Job_details.csv")
df = df.drop('Unnamed: 0', axis=1)

In [10]:
df.shape

(2015, 8)

In [19]:
df_link = pd.read_csv("Job_links_likes.csv")
df_link = df_link.drop('Unnamed: 0', axis=1)
df_link

,Url,likes,comment_num
0,https://mbasic.facebook.com/groups/jobsandinte...,4,1
1,https://mbasic.facebook.com/groups/jobsandinte...,55,3
2,https://mbasic.facebook.com/groups/jobsandinte...,13,0
3,https://mbasic.facebook.com/groups/jobsandinte...,2,0
4,https://mbasic.facebook.com/groups/jobsandinte...,24,4
...,...,...,...
3021,https://mbasic.facebook.com/groups/jobsandinte...,34,4
3022,https://mbasic.facebook.com/groups/jobsandinte...,30,2
3023,https://mbasic.facebook.com/groups/jobsandinte...,20,6
3024,https://mbasic.facebook.com/groups/jobsandinte...,14,1


In [22]:
df_link = df_link[~df_link['Url'].str.contains('https://lm.facebook.com')]

In [23]:
url_list = df_link.to_dict(orient='records')

### 爬取詳細資料

In [12]:
#抓留言
def getComment(soup):
    commentList = soup.find("div", id = "m_story_permalink_view").contents[-1].contents[0].contents[-3].contents
    commten_list = []
    if(len(commentList)>0 and commentList[0].name=='div'):
        for comment in commentList:
            if(len(comment.find_all("div"))>0):
                data = {}
                poster_a = comment.find("a")
                comment_content = comment.find_all("div")[1].text
                data['poster'] = poster_a.text
                data['posterUrl'] = url+poster_a['href']
                data['comment'] = comment_content
                commten_list.append(data)
    if(len(commten_list)>0):
        return commten_list
    else:
        return None

In [13]:
def findInfo(Url_dict):
    art_dict = {}
    try:
        driver.get(Url_dict['Url'])
    except Exception as e:
        print(e)
        art_dict['content'] = []
        return art_dict
    resp = driver.page_source
    resp = resp.replace("<br>", "。")
    soup = BeautifulSoup(resp, "html.parser")
    try:
        srt_date = soup.find("footer").find("abbr").text
    except:
        srt_date=''
    if(srt_date == ""): 
        art_dict['content'] = []
        return art_dict
    this_year = datetime.datetime.now().year
    if "小時" in srt_date:
        date_obj  = datetime.datetime.now()
    if "昨天" in srt_date:
        date_obj  = datetime.datetime.now()
    elif "年" in srt_date:
        if "上午" in srt_date:
            date_obj = datetime.datetime.strptime(srt_date, "%Y年%m月%d日上午%H:%M")
        elif "下午" in srt_date:
            date_obj = datetime.datetime.strptime(srt_date, "%Y年%m月%d日下午%H:%M")
        else:
            date_obj = datetime.datetime.strptime(srt_date, "%Y年%m月%d日")
    else:
        if "上午" in srt_date:
            date_obj = datetime.datetime.strptime(str(this_year) + "年" + srt_date, "%Y年%m月%d日上午%H:%M")
        elif "下午" in srt_date:
            date_obj = datetime.datetime.strptime(str(this_year) + "年" + srt_date, "%Y年%m月%d日下午%H:%M")

    art_dict['date'] = date_obj.strftime("%Y-%m-%d")
    # 內容
    texts_html = soup.find("div", id = "m_story_permalink_view").select_one('div > div').select('div > div')[1].select('div > div')[1].text
    art_dict['Url'] = Url_dict['Url']
    art_dict['content'] = texts_html
    art_dict['reaction_like_count'] = Url_dict['likes']
    art_dict['comment_num'] = Url_dict['comment_num']
    poster = soup.find("header").find("header").find("h3").find("a")
    art_dict['poster'] = poster.text
    art_dict['posterUrl'] = url+poster['href']
    art_dict['comments'] = getComment(soup)

    return art_dict

In [45]:
result_detail = []
for index, art_dict in enumerate(url_list):
    # print(art_dict['artUrl'])
    article = findInfo(art_dict)
    if(len(article['content'])>2):
        result_detail.append(article)

In [46]:
pd.DataFrame(result_detail).to_csv("Job_details.csv",encoding='utf-8-sig')

In [47]:
df = pd.read_csv("Job_details.csv")
df = df.drop('Unnamed: 0', axis=1)

In [48]:
df

,date,Url,content,reaction_like_count,comment_num,poster,posterUrl,comments
0,2023-04-27,https://mbasic.facebook.com/groups/jobsandinte...,哈囉～幫現在的公司招募實習生，本人心得在工作資訊下方！。 沂動行銷公司 徵求商務拓展部門實習...,4,1,Hope Hsueh,https://mbasic.facebook.com/hopegenius?eav=AfY...,"[{'poster': '林裕棠', 'posterUrl': 'https://mbasi..."
1,2023-04-29,https://mbasic.facebook.com/groups/jobsandinte...,代朋友PO #徵才 訊息󠀠󠀠【公司名稱】Beta Financehttps://www.be...,55,3,游祖鈞,https://mbasic.facebook.com/tzuchun42?eav=Afbw...,"[{'poster': '歐陽廷', 'posterUrl': 'https://mbasi..."
2,2023-04-27,https://mbasic.facebook.com/groups/jobsandinte...,【實習】Quantum International Corp (QIC) 2023-2024...,13,0,陳柏諭,https://mbasic.facebook.com/Yu.Chen.890711?eav...,NaN
3,2023-04-29,https://mbasic.facebook.com/groups/jobsandinte...,#徵才 投顧公司徵特助、財務、人事行政🙋✨歡迎加入TAROBO✨☉公司名稱：大拇哥證券投資顧...,2,0,蘇輗,https://mbasic.facebook.com/niyessu?eav=AfZKua...,NaN
4,2023-04-27,https://mbasic.facebook.com/groups/jobsandinte...,(代po)【Position Title】PChome Corporate Developm...,24,4,彭寶萱,https://mbasic.facebook.com/pengpaohsuan?eav=A...,"[{'poster': '司惟之', 'posterUrl': 'https://mbasi..."
...,...,...,...,...,...,...,...,...
2010,2021-05-10,https://mbasic.facebook.com/groups/jobsandinte...,Hi！。 元大投顧的總經研究團隊正在招募實習生。 需求時段：ㄧ、三（全天），二、五（早上）。...,60,10,吳昌樺,https://mbasic.facebook.com/profile.php?id=100...,"[{'poster': '范軒榮', 'posterUrl': 'https://mbasi..."
2011,2021-05-13,https://mbasic.facebook.com/groups/jobsandinte...,Hi大家！。 食品商Aunt Stella要徵EC囉～同事們很善良好相處，辦公室很美（？）應...,32,2,Yi Fan Shih,https://mbasic.facebook.com/yifan.shih?eav=AfY...,"[{'poster': '萬家瑜', 'posterUrl': 'https://mbasi..."
2012,2021-05-12,https://mbasic.facebook.com/groups/jobsandinte...,(代po 正職) 。 【國立陽明交通大學醫學院 陳震寰院長徵求專任研究助理】。 【計畫名稱】...,9,0,王方均,https://mbasic.facebook.com/profile.php?id=100...,NaN
2013,2021-05-11,https://mbasic.facebook.com/groups/jobsandinte...,【公司名稱】：國泰金控。 【部門名稱】：數位數據暨科技發展中心-國泰金融創新實驗室。 【工作...,46,2,李大慶,https://mbasic.facebook.com/jason.li.3745?eav=...,"[{'poster': '鄭惟娟', 'posterUrl': 'https://mbasi..."


In [44]:
result = []
for article in articles:
    data = {}
    try:
        tmplink = article.select(".dn .ds a:not([class]):not([data-ft])")[-1:][0]['href']
    
        if("mbasic" not in tmplink):
            tmplink = 'https://mbasic.facebook.com'+tmplink
    except:
        tmplink = None
    # likes = int((article.find("footer").find_all("div", class_ = "cp cq")[-1:][0].select("a[aria-label]"))[0].text)
    like_str = article.find("footer").find_all("div", class_ = "cp cq")[-1:][0].find("span", class_ = "cp cq").text
    like_str = re.findall(r'\d+', like_str)
    if(len(like_str) > 0) : 
        like_str=int(like_str[0]) 
    else: 
        like_str = 0
    # 留言
    comment_num = (article.find("footer").find_all("div", class_ = "cp cq")[-1:][0].select('a.dv:not([class*=" "])'))[0].text
    comment_num = re.findall(r'\d+', comment_num)
    if(len(comment_num) > 0) : 
        comment_num=int(comment_num[0]) 
    else: 
        comment_num = 0
    data['Url'] = tmplink
    data['likes'] = like_str
    data['comment_num'] = comment_num
    if tmplink!=None:
        result.append(data)